<a href="https://colab.research.google.com/github/wanghaiy2018/AC0499_2024/blob/main/Installation/Ollama_sample12-14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json

# Define the endpoint and payload


myip="http://localhost"

url = myip+":11434/api/generate"
payload = {
    "model": "llama3.2",
#    "model": "llama3.3",
#    "model": "qwq",
#    "model":  "hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF",

    "prompt": "math  quandratic formula",
    "stream": False
}
headers = {
    "Content-Type": "application/json"
}

# Send the POST request
try:
    response = requests.post(url, headers=headers, json=payload, timeout=30000)
    response.raise_for_status()  # Raise an HTTPError for bad responses
    print("Response:", response.json())
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


Response: {'model': 'llama3.2', 'created_at': '2024-12-16T18:43:10.7731551Z', 'response': "The quadratic formula is:\n\nx = (-b ± √(b^2 - 4ac)) / 2a\n\nWhere a, b, and c are the coefficients of the quadratic equation in the form ax^2 + bx + c = 0.\n\nHere's how it works:\n\n1. Plug in the values of a, b, and c into the formula.\n2. Calculate the value inside the square root (b^2 - 4ac).\n3. Simplify the expression to get two possible solutions for x.\n4. Choose the correct solution based on the context of the problem.\n\nExample:\n\nSuppose we have the quadratic equation x^2 + 5x + 6 = 0\n\nIn this case, a = 1, b = 5, and c = 6.\n\nUsing the quadratic formula, we get:\n\nx = (-5 ± √(5^2 - 4*1*6)) / 2*1\nx = (-5 ± √(25 - 24)) / 2\nx = (-5 ± √1) / 2\n\nSimplifying further, we get two possible solutions:\n\nx = (-5 + 1) / 2 = -2\nx = (-5 - 1) / 2 = -3\n\nTherefore, the solutions to the equation x^2 + 5x + 6 = 0 are x = -2 and x = -3.\n\nNote that the quadratic formula will always give you

In [2]:
import requests
import json

# Class to interact with the Llama model server
class LlamaLLM:
    def __init__(self, url):
        self.url = url

    def generate(self, prompt):
        payload = {
            "model": "llama3.2",
            "prompt": prompt,
            "stream": False
        }
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.post(self.url, headers=headers, data=json.dumps(payload),timeout=30000)
        response.raise_for_status()
        return response.json()['response']

# Environment class remains the same
class Environment:
    def __init__(self):
        self.state = 'initial_state'  # Initial state
        self.goal_achieved = False  # Goal not achieved
        self.result = None  # Store result

    def get_state(self):
        return self.state  # Get current state

    def change_state(self, result):
        self.state = 'goal_state'  # Change to goal state
        self.result = result  # Store result

# ReActAgent class modified to use LlamaLLM for reasoning
class ReActAgent:
    def __init__(self, environment, llm, max_attempts=5):
        self.environment = environment  # Initialize environment
        self.llm = llm  # Llama 3.1 model
        self.max_attempts = max_attempts  # Max attempts

    def perceive(self):
        return self.environment.get_state()  # Perceive environment state

    def reason(self, state):
        # Causal reasoning prompt based on paper example
        prompt = """
        Given the following causal relationships:
        - A causes B
        - B causes C
        - A is independent of C given B

        Question: Does A directly affect C?
        """
        response = self.llm.generate(prompt).strip()
        print(f"Raw Llama Response: {response}")  # Debugging: print raw response
        if 'No' in response or 'independent' in response:
            return 'achieve_goal', "A does not directly affect C"
        return 'take_action', None

    def act(self, action, result):
        if action == 'achieve_goal':
            self.environment.goal_achieved = True  # Achieve goal
            self.environment.change_state(result)  # Change state and store result

# Initialize the Llama model with the server URL


myip="http://localhost"

llm = LlamaLLM(url=myip+":11434/api/generate")

# Create environment and agent
env = Environment()
agent = ReActAgent(env, llm)

# Agent perception, reasoning, and action loop
attempts = 0
while not env.goal_achieved and attempts < agent.max_attempts:
    state = agent.perceive()
    action, result = agent.reason(state)
    agent.act(action, result)
    attempts += 1
    print(f"State: {state}, Action: {action}, Result: {result}, Attempts: {attempts}")

if env.goal_achieved:
    print(f"Goal achieved with result: {env.result}")
else:
    print("Failed to achieve goal within the maximum number of attempts.")

Raw Llama Response: Based on the given causal relationships, we can determine that A does not directly affect C.

The relationship between A and C is indirect. A causes B, and then B causes C. However, it's specified that A is independent of C given B, which means that knowing whether B has occurred (i.e., knowing whether A caused B) does not provide any additional information about the probability or effect of A on C.

In other words, even if we know that A caused B, we cannot conclude that this will lead to an effect on C. The relationship between A and C is conditioned on the occurrence of B, so it's mediated by B, rather than being a direct cause-and-effect relationship between A and C.
State: initial_state, Action: achieve_goal, Result: A does not directly affect C, Attempts: 1
Goal achieved with result: A does not directly affect C
